# 肺癌のbinary classification datasetを利用したEnsemble MLの実験とSMOTEを用いた精度改善

In [13]:
import random

from imblearn.over_sampling import SMOTE
import pandas as pd
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier

In [14]:
random.seed(0)

# Load Model
sklearnに実装されている肺癌のpositve or negativeのbinary classificationを利用した

In [15]:
dataset = load_breast_cancer(as_frame=True)
data, target = dataset.data, dataset.target

test_index = list()
for c in target.unique():
    test_index += random.sample(
        target[target == c].index.tolist(),
        int(len(data) * 0.1),
    )

x_train = data.drop(test_index, axis=0)
y_train = target.drop(test_index, axis=0)
x_test = data.iloc[test_index]
y_test = target.iloc[test_index]

# Random Forestで分類
そのままでもかなり精度は高い

In [16]:
clf = RandomForestClassifier(n_estimators=256, max_depth=None, random_state=0)
clf.fit(x_train, y_train)
print(clf.score(x_test, y_test))

0.9285714285714286


## Feature Importance
worst perimeter, area, radius 等が高くなっており，単純に癌の大きさ自体が結果に影響しているため，正しい結果と言える．

In [17]:
def feature_importance(features, importances):
    feature_imp = pd.DataFrame({
        "feature": features,
        "importance": importances,
    })
    feature_imp.sort_values("importance", axis=0, ascending=False, inplace=True)
    return feature_imp.head(5)

feature_importance(data.columns, clf.feature_importances_)

,feature,importance
22,worst perimeter,0.130424
27,worst concave points,0.124808
20,worst radius,0.122621
7,mean concave points,0.110492
23,worst area,0.102110


# クラスの分布
0よりも1の方が多い

In [18]:
y_train.value_counts()

1    301
0    156
Name: target, dtype: int64

# SMOTEによるData Augmentation -> Random Forest
- 学習データにおけるクラス間の差が2倍のほどあったが，SMOTEを適用することにより，クラス間の差を擬似的になくしている．
- 結果的に，分類の精度は向上した (0.9285 -> 0.9553  +0.268)
- feature importanceの観点から考察すると，SMOTEを適用することで，より特定の特徴量に頼って分類するようになることがわかる

In [19]:
smt = SMOTE(random_state=0)
x_train_smt, y_train_smt = smt.fit_resample(x_train, y_train)

In [20]:
clf.fit(x_train_smt, y_train_smt)
print(clf.score(x_test, y_test))
feature_importance(data.columns, clf.feature_importances_)

0.9553571428571429


,feature,importance
22,worst perimeter,0.135527
7,mean concave points,0.134572
27,worst concave points,0.123823
23,worst area,0.102645
20,worst radius,0.094060


# より不均衡なデータに対するSMOTEの威力を調査
- データをより不均衡にした場合，Data Augmentationを適用しないと著しく精度が低下した．
- SMOTEを適用した場合でも精度は劣化するが，適用しない場合と比べて精度が向上している．
- また，SMOTEを適用することで，モデルがより有用な特徴に頼ろうとする傾向も見られた

## 学習データをあえて不均衡にする
- minor classである0のデータを0.5カットしている

In [21]:
minor_class = 0
rm_rate = 0.75

minor_index = y_train[y_train == minor_class].index.tolist()
rm_index = random.sample(minor_index, int(len(minor_index) * rm_rate))
x_train_imb = x_train.drop(rm_index, axis=0)
y_train_imb = y_train.drop(rm_index, axis=0)
y_train_imb.value_counts()

1    301
0     39
Name: target, dtype: int64

## そのままRandom Forestで学習

In [22]:
clf.fit(x_train_imb, y_train_imb)
print(clf.score(x_test, y_test))
feature_importance(data.columns, clf.feature_importances_)

0.8928571428571429


,feature,importance
27,worst concave points,0.153551
7,mean concave points,0.121166
22,worst perimeter,0.099792
20,worst radius,0.097573
2,mean perimeter,0.074802


## SMOTEしてからRandom Forestで学習

In [23]:
x_train_imb_smt, y_train_imb_smt = smt.fit_resample(x_train_imb, y_train_imb)
y_train_imb_smt.value_counts()

1    301
0    301
Name: target, dtype: int64

In [25]:
clf.fit(x_train_imb_smt, y_train_imb_smt)
print(clf.score(x_test, y_test))
feature_importance(data.columns, clf.feature_importances_)

0.9196428571428571


,feature,importance
27,worst concave points,0.161736
7,mean concave points,0.129310
6,mean concavity,0.090461
22,worst perimeter,0.084780
23,worst area,0.083245


# まとめ
- 肺癌のpos, negを判定するbinary classificationのデータセットを利用した実験を行った
- RandomForestによる学習による精度の検証を行った
- SMOTEを適用することによるモデルの精度改善についても調査を行った
- 今回使用したデータセットに対しては，SMOTEにより平均2%ほどの精度改善が見られた
- また，SMOTEを適用することにより，モデルがより有用な特徴量をに依存して分類する傾向にあることがわかった